# 32-网络请求处理

## 📚 用途说明

**学习目标**：
- 掌握Python网络请求的基本概念和方法
- 熟练使用requests库进行HTTP操作
- 理解RESTful API的交互模式和认证机制
- 能够处理网络异常、重试和限流等实际问题

**前置要求**：
- 已完成31-JSON与XML处理学习
- 熟练掌握Python字典、列表等数据结构
- 了解JSON/XML数据格式和异常处理机制

**与LangChain关联**：
- 网络请求是LangChain API集成的基础
- HTTP客户端模式影响LangChain的LLM API调用
- 认证和错误处理对LangChain的稳定性至关重要
- 为后续学习LangChain的API工具和集成做准备

---

## 🔢 知识点覆盖

### 4.5 网络请求处理 [⭐⭐进阶]
**知识点说明**：网络请求处理是现代编程的重要技能，特别是在API集成、数据获取、微服务通信等场景。requests库提供了简洁而强大的HTTP客户端功能，是Python网络编程的事实标准。

**学习要求**：
- 掌握HTTP协议的基本概念和方法
- 熟练使用requests库进行各种HTTP操作
- 理解API认证和会话管理机制
- 能够处理网络异常、重试和限流问题

**案例要求**：
- 实现完整的HTTP客户端应用示例
- 进行API交互和数据处理练习
- 应用网络编程解决实际问题
- 验证点：能独立构建稳定的API客户端

In [ ]:
print("🌐 HTTP基础概念:")
print("=" * 50)

import urllib.request
import urllib.parse
import urllib.error
import json
import time
from typing import Dict, List, Any, Optional, Union
from dataclasses import dataclass
from enum import Enum
import hashlib
import base64

# 1. HTTP协议基础
print(f"📝 1. HTTP协议基础:")

# 1.1 HTTP方法
print(f"\n   🔧 1.1 HTTP方法:")

class HTTPMethod(Enum):
    """HTTP方法枚举"""
    GET = "GET"
    POST = "POST"
    PUT = "PUT"
    DELETE = "DELETE"
    PATCH = "PATCH"
    HEAD = "HEAD"
    OPTIONS = "OPTIONS"

http_methods_info = {
    HTTPMethod.GET: "获取资源，安全且幂等",
    HTTPMethod.POST: "创建资源，非幂等",
    HTTPMethod.PUT: "更新/替换资源，幂等",
    HTTPMethod.DELETE: "删除资源，幂等",
    HTTPMethod.PATCH: "部分更新资源，非幂等",
    HTTPMethod.HEAD: "获取响应头，无响应体",
    HTTPMethod.OPTIONS: "获取服务器支持的HTTP方法"
}

print(f"   HTTP方法说明:")
for method, description in http_methods_info.items():
    print(f"     {method.value:8} - {description}")

# 1.2 HTTP状态码
print(f"\n   📊 1.2 HTTP状态码:")

@dataclass
class StatusCodeInfo:
    """状态码信息"""
    code: int
    category: str
    meaning: str
    common_codes: List[str]

status_codes = [
    StatusCodeInfo(200, "成功", "请求成功", ["200 OK", "201 Created", "204 No Content"]),
    StatusCodeInfo(300, "重定向", "需要进一步操作", ["301 Moved Permanently", "302 Found", "304 Not Modified"]),
    StatusCodeInfo(400, "客户端错误", "请求存在错误", ["400 Bad Request", "401 Unauthorized", "403 Forbidden", "404 Not Found"]),
    StatusCodeInfo(500, "服务器错误", "服务器处理错误", ["500 Internal Server Error", "502 Bad Gateway", "503 Service Unavailable"])
]

print(f"   HTTP状态码分类:")
for status in status_codes:
    print(f"\n     {status.code}xx - {status.category}:")
    print(f"       含义: {status.meaning}")
    print(f"       常见: {', '.join(status.common_codes)}")

# 1.3 URL结构
print(f"\n   🔗 1.3 URL结构:")

def parse_url_structure(url: str) -> Dict[str, str]:
    """解析URL结构"""
    parsed = urllib.parse.urlparse(url)
    return {
        "协议": parsed.scheme,
        "域名": parsed.netloc,
        "路径": parsed.path,
        "参数": parsed.params,
        "查询": parsed.query,
        "片段": parsed.fragment
    }

example_urls = [
    "https://api.example.com/v1/users?page=1&limit=10#results",
    "http://localhost:8080/api/data;type=json?format=pretty",
    "ftp://files.example.com/downloads/file.zip"
]

print(f"   URL结构解析示例:")
for url in example_urls:
    print(f"\n     URL: {url}")
    structure = parse_url_structure(url)
    for part, value in structure.items():
        if value:
            print(f"       {part}: {value}")

# 2. urllib基础使用
print(f"\n📝 2. urllib基础使用:")

# 2.1 基本GET请求
print(f"\n   🔍 2.1 基本GET请求:")

def make_simple_get_request(url: str) -> Optional[str]:
    """发送简单GET请求"""
    try:
        with urllib.request.urlopen(url) as response:
            return response.read().decode('utf-8')
    except urllib.error.URLError as e:
        print(f"   请求失败: {e}")
        return None
    except Exception as e:
        print(f"   其他错误: {e}")
        return None

# 使用公共API进行测试 (JSONPlaceholder)
test_url = "https://jsonplaceholder.typicode.com/posts/1"
print(f"   测试URL: {test_url}")

# 注意：这里只是演示代码结构，实际运行可能需要网络连接
print(f"   GET请求示例代码已准备")
print(f"   # response = make_simple_get_request(test_url)")
print(f"   # print(f'响应内容: {response}')")

# 2.2 带参数的请求
print(f"\n   📋 2.2 带参数的请求:")

def build_url_with_params(base_url: str, params: Dict[str, str]) -> str:
    """构建带参数的URL"""
    query_string = urllib.parse.urlencode(params)
    separator = '&' if '?' in base_url else '?'
    return f"{base_url}{separator}{query_string}"

base_api_url = "https://jsonplaceholder.typicode.com/posts"
request_params = {
    "userId": "1",
    "_limit": "5",
    "_sort": "id",
    "_order": "desc"
}

full_url = build_url_with_params(base_api_url, request_params)
print(f"   基础URL: {base_api_url}")
print(f"   参数: {request_params}")
print(f"   完整URL: {full_url}")

# 2.3 POST请求
print(f"\n   📤 2.3 POST请求:")

def make_post_request(url: str, data: Dict[str, Any]) -> Optional[str]:
    """发送POST请求"""
    try:
        # 编码数据
        json_data = json.dumps(data).encode('utf-8')
        
        # 创建请求对象
        req = urllib.request.Request(
            url,
            data=json_data,
            headers={'Content-Type': 'application/json'}
        )
        
        with urllib.request.urlopen(req) as response:
            return response.read().decode('utf-8')
    except urllib.error.URLError as e:
        print(f"   POST请求失败: {e}")
        return None

# POST数据示例
post_data = {
    "title": "测试文章",
    "body": "这是一篇测试文章的内容",
    "userId": 1
}

print(f"   POST数据示例: {json.dumps(post_data, ensure_ascii=False, indent=2)}")
print(f"   POST请求示例代码已准备")
print(f"   # response = make_post_request(base_api_url, post_data)")

# 2.4 请求头设置
print(f"\n   📋 2.4 请求头设置:")

def create_request_with_headers(url: str, headers: Dict[str, str]) -> urllib.request.Request:
    """创建带自定义头的请求"""
    return urllib.request.Request(url, headers=headers)

# 常用请求头
common_headers = {
    "User-Agent": "Python-Learning-Client/1.0",
    "Accept": "application/json",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8",
    "Connection": "keep-alive",
    "Content-Type": "application/json"
}

print(f"   常用请求头:")
for key, value in common_headers.items():
    print(f"     {key}: {value}")

print(f"\n✅ HTTP基础概念完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握HTTP协议的基本概念和方法")
print(f"   ✓ 理解HTTP状态码的含义和分类")
print(f"   ✓ 能够解析和构建URL结构")
print(f"   ✓ 掌握urllib的基本使用方法")

### requests库进阶应用 [⭐⭐进阶]
**知识点说明**：requests库是Python中最流行的HTTP客户端库，提供了简洁而强大的API。掌握requests库对于现代Web开发、API集成、数据获取等实际应用场景非常重要。

**学习要求**：
- 掌握requests库的核心功能和API
- 熟练进行各种HTTP请求操作
- 理解会话管理和认证机制
- 能够处理复杂的网络请求场景

**案例要求**：
- 实现完整的requests应用示例
- 进行API交互和数据处理练习
- 应用requests解决实际问题
- 验证点：能独立构建功能完善的HTTP客户端

In [ ]:
print("🔐 API认证与错误处理:")
print("=" * 50)

import json
import time
import hashlib
import base64
import hmac
from typing import Dict, List, Any, Optional, Union, Callable
from dataclasses import dataclass
from enum import Enum
from datetime import datetime, timedelta
import random

# 1. API认证方式
print(f"📝 1. API认证方式:")

# 1.1 API Key认证
print(f"\n   🔑 1.1 API Key认证:")

class APIKeyAuth:
    """API Key认证处理器"""
    
    def __init__(self, api_key: str, header_name: str = "X-API-Key"):
        self.api_key = api_key
        self.header_name = header_name
    
    def get_headers(self) -> Dict[str, str]:
        """获取认证头"""
        return {self.header_name: self.api_key}
    
    def add_to_params(self, params: Dict = None) -> Dict[str, str]:
        """添加到请求参数"""
        if params is None:
            params = {}
        params["api_key"] = self.api_key
        return params

# 1.2 Bearer Token认证
print(f"\n   🎫 1.2 Bearer Token认证:")

class BearerTokenAuth:
    """Bearer Token认证处理器"""
    
    def __init__(self, token: str):
        self.token = token
    
    def get_headers(self) -> Dict[str, str]:
        """获取认证头"""
        return {"Authorization": f"Bearer {self.token}"}
    
    def is_expired(self, expires_at: datetime) -> bool:
        """检查token是否过期"""
        return datetime.now() >= expires_at

# 1.3 Basic认证
print(f"\n   🔒 1.3 Basic认证:")

class BasicAuth:
    """Basic认证处理器"""
    
    def __init__(self, username: str, password: str):
        self.username = username
        self.password = password
    
    def get_headers(self) -> Dict[str, str]:
        """获取认证头"""
        credentials = f"{self.username}:{self.password}"
        encoded_credentials = base64.b64encode(credentials.encode()).decode()
        return {"Authorization": f"Basic {encoded_credentials}"}

# 1.4 HMAC签名认证
print(f"\n   🖋️ 1.4 HMAC签名认证:")

class HMACAuth:
    """HMAC签名认证处理器"""
    
    def __init__(self, api_key: str, api_secret: str):
        self.api_key = api_key
        self.api_secret = api_secret
    
    def generate_signature(self, method: str, url: str, params: Dict = None, body: str = "") -> str:
        """生成HMAC签名"""
        # 构建签名字符串
        timestamp = str(int(time.time()))
        
        if params:
            # 按字母顺序排序参数
            sorted_params = sorted(params.items())
            query_string = "&".join([f"{k}={v}" for k, v in sorted_params])
        else:
            query_string = ""
        
        sign_string = f"{method}\n{url}\n{query_string}\n{body}\n{timestamp}"
        
        # 生成签名
        signature = hmac.new(
            self.api_secret.encode(),
            sign_string.encode(),
            hashlib.sha256
        ).hexdigest()
        
        return signature
    
    def get_headers(self, method: str, url: str, params: Dict = None, body: str = "") -> Dict[str, str]:
        """获取认证头"""
        timestamp = str(int(time.time()))
        signature = self.generate_signature(method, url, params, body)
        
        return {
            "X-API-Key": self.api_key,
            "X-Timestamp": timestamp,
            "X-Signature": signature
        }

# 认证方式演示
print(f"   认证方式演示:")

# API Key认证
api_key_auth = APIKeyAuth("sk-1234567890abcdef")
print(f"\n   API Key认证头: {api_key_auth.get_headers()}")

# Bearer Token认证
bearer_auth = BearerTokenAuth("eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9")
print(f"   Bearer Token认证头: {bearer_auth.get_headers()}")

# Basic认证
basic_auth = BasicAuth("admin", "password123")
print(f"   Basic认证头: {basic_auth.get_headers()}")

# HMAC认证
hmac_auth = HMACAuth("api_key_123", "api_secret_456")
hmac_headers = hmac_auth.get_headers("GET", "/api/data", {"limit": "10"})
print(f"   HMAC认证头: {hmac_headers}")

# 2. 错误处理机制
print(f"\n📝 2. 错误处理机制:")

# 2.1 异常类型定义
print(f"\n   ⚠️ 2.1 异常类型定义:")

class APIException(Exception):
    """API异常基类"""
    pass

class AuthenticationError(APIException):
    """认证错误"""
    pass

class RateLimitError(APIException):
    """限流错误"""
    def __init__(self, message: str, retry_after: int = None):
        super().__init__(message)
        self.retry_after = retry_after

class ServerError(APIException):
    """服务器错误"""
    pass

class NetworkError(APIException):
    """网络错误"""
    pass

class TimeoutError(APIException):
    """超时错误"""
    pass

# 2.2 错误处理器
print(f"\n   🔧 2.2 错误处理器:")

@dataclass
class ErrorResponse:
    """错误响应数据结构"""
    status_code: int
    error_type: str
    message: str
    details: Dict[str, Any] = None
    timestamp: datetime = None

class ErrorHandler:
    """错误处理器"""
    
    @staticmethod
    def handle_http_error(status_code: int, response_text: str) -> APIException:
        """处理HTTP错误"""
        try:
            error_data = json.loads(response_text)
            message = error_data.get('message', f'HTTP {status_code} Error')
        except json.JSONDecodeError:
            message = f'HTTP {status_code} Error'
        
        if status_code == 401:
            return AuthenticationError(message)
        elif status_code == 429:
            retry_after = error_data.get('retry_after', 60)
            return RateLimitError(message, retry_after)
        elif 500 <= status_code < 600:
            return ServerError(message)
        elif 400 <= status_code < 500:
            return APIException(message)
        else:
            return APIException(message)
    
    @staticmethod
    def handle_network_error(error: Exception) -> NetworkError:
        """处理网络错误"""
        if "timeout" in str(error).lower():
            return TimeoutError(f"网络超时: {error}")
        else:
            return NetworkError(f"网络错误: {error}")

# 2.3 重试机制
print(f"\n   🔄 2.3 重试机制:")

class RetryStrategy:
    """重试策略"""
    
    def __init__(self, max_retries: int = 3, backoff_factor: float = 1.0, retry_on: List[type] = None):
        self.max_retries = max_retries
        self.backoff_factor = backoff_factor
        self.retry_on = retry_on or [NetworkError, ServerError, TimeoutError]
    
    def should_retry(self, exception: Exception, attempt: int) -> bool:
        """判断是否应该重试"""
        if attempt >= self.max_retries:
            return False
        
        return any(isinstance(exception, error_type) for error_type in self.retry_on)
    
    def get_delay(self, attempt: int) -> float:
        """获取重试延迟时间"""
        return self.backoff_factor * (2 ** attempt)

def with_retry(strategy: RetryStrategy = None):
    """重试装饰器"""
    if strategy is None:
        strategy = RetryStrategy()
    
    def decorator(func: Callable):
        def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(strategy.max_retries + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    
                    if not strategy.should_retry(e, attempt):
                        break
                    
                    delay = strategy.get_delay(attempt)
                    print(f"   请求失败，{delay:.1f}秒后重试 (第{attempt + 1}次): {e}")
                    time.sleep(delay)
            
            raise last_exception
        
        return wrapper
    
    return decorator

# 3. 完整API客户端
print(f"\n📝 3. 完整API客户端:")

class APIClient:
    """完整的API客户端"""
    
    def __init__(self, base_url: str, auth: Any = None, timeout: float = 30.0):
        self.base_url = base_url.rstrip('/')
        self.auth = auth
        self.timeout = timeout
        self.session_headers = {
            "User-Agent": "Python-Learning-Client/1.0",
            "Accept": "application/json",
            "Content-Type": "application/json"
        }
        self.error_handler = ErrorHandler()
        self.retry_strategy = RetryStrategy(max_retries=3, backoff_factor=1.0)
    
    def _prepare_headers(self, additional_headers: Dict = None) -> Dict[str, str]:
        """准备请求头"""
        headers = self.session_headers.copy()
        
        if additional_headers:
            headers.update(additional_headers)
        
        if self.auth:
            auth_headers = self.auth.get_headers()
            headers.update(auth_headers)
        
        return headers
    
    def _handle_response(self, status_code: int, response_text: str) -> Any:
        """处理响应"""
        if status_code >= 400:
            raise self.error_handler.handle_http_error(status_code, response_text)
        
        try:
            return json.loads(response_text)
        except json.JSONDecodeError:
            return response_text
    
    def _validate_json_payload(self, data: Dict) -> bool:
        """验证JSON载荷数据 - LangChain风格的数据验证"""
        # 基础字段验证
        if not isinstance(data, dict):
            raise ValueError("请求数据必须为字典类型")
        
        # 检查必需字段（示例：标题字段）
        required_fields = ["title"]  # 可根据实际API需求调整
        for field in required_fields:
            if field not in data or not data[field]:
                raise ValueError(f"缺少必需字段: {field}")
        
        # 数据类型验证
        if "value" in data and not isinstance(data["value"], (int, float)):
            raise ValueError("value字段必须为数字类型")
        
        # 字符串长度限制
        if "title" in data and len(data["title"]) > 200:
            raise ValueError("标题长度不能超过200字符")
        
        # 数值范围验证
        if "value" in data and not (0 <= data["value"] <= 1000000):
            raise ValueError("value字段必须在0-1000000范围内")
        
        print(f"   ✅ JSON数据验证通过")
        return True
    
    @with_retry()
    def get(self, endpoint: str, params: Dict = None, **kwargs) -> Any:
        """发送GET请求"""
        url = f"{self.base_url}/{endpoint.lstrip('/')}"
        headers = self._prepare_headers(kwargs.get('headers'))
        
        # 模拟请求
        print(f"   发送GET请求: {url}")
        print(f"   请求头: {headers}")
        if params:
            print(f"   请求参数: {params}")
        
        # 模拟响应
        if "protected" in endpoint and not self.auth:
            return self._handle_response(401, '{"error": "需要认证"}')
        elif "rate-limit" in endpoint:
            return self._handle_response(429, '{"error": "请求过于频繁", "retry_after": 60}')
        elif "server-error" in endpoint:
            return self._handle_response(500, '{"error": "服务器内部错误"}')
        else:
            mock_data = {"message": "请求成功", "timestamp": datetime.now().isoformat()}
            return self._handle_response(200, json.dumps(mock_data, ensure_ascii=False))
    
    @with_retry()
    def post(self, endpoint: str, data: Dict = None, **kwargs) -> Any:
        """发送POST请求"""
        url = f"{self.base_url}/{endpoint.lstrip('/')}"
        headers = self._prepare_headers(kwargs.get('headers'))
        
        print(f"   发送POST请求: {url}")
        print(f"   请求数据: {data}")
        
        # JSON数据验证 - LangChain风格的数据预处理
        if data:
            self._validate_json_payload(data)
            created_data = {"id": random.randint(100, 999), **data}
            return self._handle_response(201, json.dumps(created_data, ensure_ascii=False))
        else:
            return self._handle_response(400, '{"error": "请求数据为空"}')
    
    def set_rate_limit(self, requests_per_second: int):
        """设置速率限制"""
        self.rate_limit = requests_per_second
        self.last_request_time = 0
    
    def _enforce_rate_limit(self):
        """执行速率限制"""
        if hasattr(self, 'rate_limit'):
            current_time = time.time()
            time_since_last = current_time - self.last_request_time
            min_interval = 1.0 / self.rate_limit
            
            if time_since_last < min_interval:
                sleep_time = min_interval - time_since_last
                print(f"   速率限制: 等待 {sleep_time:.2f} 秒")
                time.sleep(sleep_time)
            
            self.last_request_time = time.time()

# API客户端使用示例
print(f"\n   API客户端使用示例:")

# 创建客户端
client = APIClient("https://api.example.com/v1")

# 测试无认证请求
try:
    response = client.get("public/data")
    print(f"   公共API响应: {response}")
except Exception as e:
    print(f"   请求失败: {e}")

# 测试需要认证的请求
try:
    response = client.get("protected/data")
except AuthenticationError as e:
    print(f"   认证错误: {e}")

# 使用认证的客户端
auth_client = APIClient(
    "https://api.example.com/v1",
    auth=BearerTokenAuth("your-bearer-token")
)

try:
    response = auth_client.get("protected/data")
    print(f"   认证API响应: {response}")
except Exception as e:
    print(f"   请求失败: {e}")

# 测试POST请求（带数据验证）
try:
    new_data = {"name": "测试项目", "value": 123}
    response = auth_client.post("items", data=new_data)
    print(f"   创建结果: {response}")
except Exception as e:
    print(f"   创建失败: {e}")

# 测试数据验证
try:
    invalid_data = {"value": "invalid"}  # 缺少title字段
    response = auth_client.post("items", data=invalid_data)
except ValueError as e:
    print(f"   数据验证错误: {e}")

# 测试限流处理
try:
    response = client.get("rate-limit/test")
except RateLimitError as e:
    print(f"   限流错误: {e}, {e.retry_after}秒后重试")

print(f"\n✅ API认证与错误处理完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握常见的API认证方式")
print(f"   ✓ 理解网络异常的分类和处理")
print(f"   ✓ 能够实现重试机制和限流策略")
print(f"   ✓ 掌握构建健壮API客户端的技巧")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**4.5 网络请求处理 [⭐⭐进阶]**
- ✅ 掌握HTTP协议的基本概念和方法
- ✅ 熟练使用requests库进行各种HTTP操作
- ✅ 理解API认证和会话管理机制
- ✅ 能够处理网络异常、重试和限流问题
- ✅ 掌握urllib和requests库的核心功能
- ✅ 理解RESTful API的交互模式
- ✅ 能实现完整的API客户端类
- ✅ 掌握认证、错误处理和重试机制
- ✅ 能独立构建稳定的网络应用

### 🎯 与LangChain学习的关联

**网络请求重要性**：
- 网络请求是LangChain API集成的基础
- HTTP客户端模式影响LangChain的LLM API调用
- 认证和错误处理对LangChain的稳定性至关重要
- 限流和重试机制在LangChain的API工具中发挥重要作用

**实际应用场景**：
- LangChain的LLM集成需要稳定的HTTP客户端
- API工具和函数调用依赖网络请求处理
- 外部数据源集成需要认证和错误处理
- 流式响应和异步请求在LangChain中广泛应用

### 📚 进阶学习建议

1. **练习建议**：
   - 多练习不同API的集成和认证方式
   - 深入学习异步HTTP请求处理
   - 掌握WebSocket和实时通信技术

2. **扩展学习**：
   - 学习HTTP/2和HTTP/3协议特性
   - 了解GraphQL和REST API的差异
   - 探索微服务架构中的服务通信

3. **实际应用**：
   - 构建API网关和代理服务
   - 开发分布式系统的通信层
   - 实现云服务的集成工具

### 🔧 常见错误与注意事项

1. **认证信息泄露**：
   ```python
   # 错误：硬编码认证信息
   api_key = "sk-1234567890abcdef"  # 不要这样做
   
   # 正确：使用环境变量或配置文件
   import os
   api_key = os.getenv('API_KEY')
   ```

2. **忽略HTTPS验证**：
   ```python
   # 错误：在生产环境忽略SSL验证
   response = requests.get(url, verify=False)
   
   # 正确：使用有效的证书或指定CA包
   response = requests.get(url, verify='/path/to/ca-bundle.crt')
   ```

3. **无限重试**：
   ```python
   # 错误：没有限制的重试
   while True:
       try:
           return make_request()
       except Exception:
           time.sleep(1)
   
   # 正确：使用指数退避和最大重试次数
   for attempt in range(max_retries):
       try:
           return make_request()
       except Exception as e:
           if attempt == max_retries - 1:
               raise e
           time.sleep(2 ** attempt)
   ```

4. **阻塞主线程**：
   ```python
   # 错误：在主线程中进行长时间网络请求
   def handle_request():
       response = requests.get(url, timeout=300)  # 阻塞5分钟
       return response
   
   # 正确：使用异步请求或设置合理的超时
   def handle_request():
       response = requests.get(url, timeout=30)
       return response
   # 或使用 aiohttp 进行异步请求
   ```

5. **内存泄漏**：
   ```python
   # 错误：没有正确关闭连接
   session = requests.Session()
   # 使用session但没有关闭
   
   # 正确：使用上下文管理器
   with requests.Session() as session:
       response = session.get(url)
   ```

6. **忽略响应大小**：
   ```python
   # 错误：下载大文件时没有流式处理
   response = requests.get('https://example.com/large-file.zip')
   data = response.content  # 可能占用大量内存
   
   # 正确：使用流式下载
   with requests.get('https://example.com/large-file.zip', stream=True) as response:
       with open('file.zip', 'wb') as f:
           for chunk in response.iter_content(chunk_size=8192):
               f.write(chunk)
   ```

### 🌐 实际API测试建议

**📝 重要提示**：
本笔记本中的所有网络请求示例都使用了模拟实现（Mock），以便在没有网络连接的环境下学习。当你有网络访问权限时，强烈建议使用真实的API进行测试：

**推荐的测试API**：
1. **JSONPlaceholder** - 免费的REST API测试服务
   ```python
   # 安装requests库
   # pip install requests
   
   import requests
   
   # 真实API测试示例
   response = requests.get('https://jsonplaceholder.typicode.com/posts/1')
   print(f"真实API响应: {response.json()}")
   ```

2. **HTTPBin** - HTTP请求和响应测试服务
   ```python
   # 测试各种HTTP功能
   response = requests.get('https://httpbin.org/get', params={'key': 'value'})
   response = requests.post('https://httpbin.org/post', json={'data': 'test'})
   ```

3. **公共天气API** - 实际业务场景测试
   ```python
   # 需要注册获取API密钥
   # response = requests.get(f'https://api.openweathermap.org/data/2.5/weather?q=Beijing&appid={API_KEY}')
   ```

**测试建议**：
- 从简单的GET请求开始测试
- 逐步尝试POST、PUT、DELETE等方法
- 测试不同的认证方式
- 验证错误处理和重试机制
- 观察真实的网络延迟和限流情况

---

**🎉 恭喜完成网络请求处理学习！**

你已经掌握了Python网络编程的核心技能，能够熟练使用requests库进行HTTP操作，掌握了API认证、错误处理、重试机制等高级技术，为后续学习LangChain的API集成和工具调用奠定了坚实基础。

## 🚀 下一步学习预告

**继续第四节：常用标准库**：
- 4.6 多线程与多进程 [⭐⭐进阶]
- 4.7 数据序列化与存储 [⭐⭐进阶]
- 4.8 系统信息获取 [⭐⭐进阶]
- 4.9 常用工具模块 [⭐⭐进阶]

继续加油，完成Python标准库的学习！

In [ ]:
print("🔐 API认证与错误处理:")
print("=" * 50)

import json
import time
import hashlib
import base64
import hmac
from typing import Dict, List, Any, Optional, Union, Callable
from dataclasses import dataclass
from enum import Enum
from datetime import datetime, timedelta
import random

# 1. API认证方式
print(f"📝 1. API认证方式:")

# 1.1 API Key认证
print(f"\n   🔑 1.1 API Key认证:")

class APIKeyAuth:
    """API Key认证处理器"""
    
    def __init__(self, api_key: str, header_name: str = "X-API-Key"):
        self.api_key = api_key
        self.header_name = header_name
    
    def get_headers(self) -> Dict[str, str]:
        """获取认证头"""
        return {self.header_name: self.api_key}
    
    def add_to_params(self, params: Dict = None) -> Dict[str, str]:
        """添加到请求参数"""
        if params is None:
            params = {}
        params["api_key"] = self.api_key
        return params

# 1.2 Bearer Token认证
print(f"\n   🎫 1.2 Bearer Token认证:")

class BearerTokenAuth:
    """Bearer Token认证处理器"""
    
    def __init__(self, token: str):
        self.token = token
    
    def get_headers(self) -> Dict[str, str]:
        """获取认证头"""
        return {"Authorization": f"Bearer {self.token}"}
    
    def is_expired(self, expires_at: datetime) -> bool:
        """检查token是否过期"""
        return datetime.now() >= expires_at

# 1.3 Basic认证
print(f"\n   🔒 1.3 Basic认证:")

class BasicAuth:
    """Basic认证处理器"""
    
    def __init__(self, username: str, password: str):
        self.username = username
        self.password = password
    
    def get_headers(self) -> Dict[str, str]:
        """获取认证头"""
        credentials = f"{self.username}:{self.password}"
        encoded_credentials = base64.b64encode(credentials.encode()).decode()
        return {"Authorization": f"Basic {encoded_credentials}"}

# 1.4 HMAC签名认证
print(f"\n   🖋️ 1.4 HMAC签名认证:")

class HMACAuth:
    """HMAC签名认证处理器"""
    
    def __init__(self, api_key: str, api_secret: str):
        self.api_key = api_key
        self.api_secret = api_secret
    
    def generate_signature(self, method: str, url: str, params: Dict = None, body: str = "") -> str:
        """生成HMAC签名"""
        # 构建签名字符串
        timestamp = str(int(time.time()))
        
        if params:
            # 按字母顺序排序参数
            sorted_params = sorted(params.items())
            query_string = "&".join([f"{k}={v}" for k, v in sorted_params])
        else:
            query_string = ""
        
        sign_string = f"{method}\n{url}\n{query_string}\n{body}\n{timestamp}"
        
        # 生成签名
        signature = hmac.new(
            self.api_secret.encode(),
            sign_string.encode(),
            hashlib.sha256
        ).hexdigest()
        
        return signature
    
    def get_headers(self, method: str, url: str, params: Dict = None, body: str = "") -> Dict[str, str]:
        """获取认证头"""
        timestamp = str(int(time.time()))
        signature = self.generate_signature(method, url, params, body)
        
        return {
            "X-API-Key": self.api_key,
            "X-Timestamp": timestamp,
            "X-Signature": signature
        }

# 认证方式演示
print(f"   认证方式演示:")

# API Key认证
api_key_auth = APIKeyAuth("sk-1234567890abcdef")
print(f"\n   API Key认证头: {api_key_auth.get_headers()}")

# Bearer Token认证
bearer_auth = BearerTokenAuth("eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9")
print(f"   Bearer Token认证头: {bearer_auth.get_headers()}")

# Basic认证
basic_auth = BasicAuth("admin", "password123")
print(f"   Basic认证头: {basic_auth.get_headers()}")

# HMAC认证
hmac_auth = HMACAuth("api_key_123", "api_secret_456")
hmac_headers = hmac_auth.get_headers("GET", "/api/data", {"limit": "10"})
print(f"   HMAC认证头: {hmac_headers}")

# 2. 错误处理机制
print(f"\n📝 2. 错误处理机制:")

# 2.1 异常类型定义
print(f"\n   ⚠️ 2.1 异常类型定义:")

class APIException(Exception):
    """API异常基类"""
    pass

class AuthenticationError(APIException):
    """认证错误"""
    pass

class RateLimitError(APIException):
    """限流错误"""
    def __init__(self, message: str, retry_after: int = None):
        super().__init__(message)
        self.retry_after = retry_after

class ServerError(APIException):
    """服务器错误"""
    pass

class NetworkError(APIException):
    """网络错误"""
    pass

class TimeoutError(APIException):
    """超时错误"""
    pass

# 2.2 错误处理器
print(f"\n   🔧 2.2 错误处理器:")

@dataclass
class ErrorResponse:
    """错误响应数据结构"""
    status_code: int
    error_type: str
    message: str
    details: Dict[str, Any] = None
    timestamp: datetime = None

class ErrorHandler:
    """错误处理器"""
    
    @staticmethod
    def handle_http_error(status_code: int, response_text: str) -> APIException:
        """处理HTTP错误"""
        try:
            error_data = json.loads(response_text)
            message = error_data.get('message', f'HTTP {status_code} Error')
        except json.JSONDecodeError:
            message = f'HTTP {status_code} Error'
        
        if status_code == 401:
            return AuthenticationError(message)
        elif status_code == 429:
            retry_after = error_data.get('retry_after', 60)
            return RateLimitError(message, retry_after)
        elif 500 <= status_code < 600:
            return ServerError(message)
        elif 400 <= status_code < 500:
            return APIException(message)
        else:
            return APIException(message)
    
    @staticmethod
    def handle_network_error(error: Exception) -> NetworkError:
        """处理网络错误"""
        if "timeout" in str(error).lower():
            return TimeoutError(f"网络超时: {error}")
        else:
            return NetworkError(f"网络错误: {error}")

# 2.3 重试机制
print(f"\n   🔄 2.3 重试机制:")

class RetryStrategy:
    """重试策略"""
    
    def __init__(self, max_retries: int = 3, backoff_factor: float = 1.0, retry_on: List[type] = None):
        self.max_retries = max_retries
        self.backoff_factor = backoff_factor
        self.retry_on = retry_on or [NetworkError, ServerError, TimeoutError]
    
    def should_retry(self, exception: Exception, attempt: int) -> bool:
        """判断是否应该重试"""
        if attempt >= self.max_retries:
            return False
        
        return any(isinstance(exception, error_type) for error_type in self.retry_on)
    
    def get_delay(self, attempt: int) -> float:
        """获取重试延迟时间"""
        return self.backoff_factor * (2 ** attempt)

def with_retry(strategy: RetryStrategy = None):
    """重试装饰器"""
    if strategy is None:
        strategy = RetryStrategy()
    
    def decorator(func: Callable):
        def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(strategy.max_retries + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    
                    if not strategy.should_retry(e, attempt):
                        break
                    
                    delay = strategy.get_delay(attempt)
                    print(f"   请求失败，{delay:.1f}秒后重试 (第{attempt + 1}次): {e}")
                    time.sleep(delay)
            
            raise last_exception
        
        return wrapper
    
    return decorator

# 3. 完整API客户端
print(f"\n📝 3. 完整API客户端:")

class APIClient:
    """完整的API客户端"""
    
    def __init__(self, base_url: str, auth: Any = None, timeout: float = 30.0):
        self.base_url = base_url.rstrip('/')
        self.auth = auth
        self.timeout = timeout
        self.session_headers = {
            "User-Agent": "Python-Learning-Client/1.0",
            "Accept": "application/json",
            "Content-Type": "application/json"
        }
        self.error_handler = ErrorHandler()
        self.retry_strategy = RetryStrategy(max_retries=3, backoff_factor=1.0)
    
    def _prepare_headers(self, additional_headers: Dict = None) -> Dict[str, str]:
        """准备请求头"""
        headers = self.session_headers.copy()
        
        if additional_headers:
            headers.update(additional_headers)
        
        if self.auth:
            auth_headers = self.auth.get_headers()
            headers.update(auth_headers)
        
        return headers
    
    def _handle_response(self, status_code: int, response_text: str) -> Any:
        """处理响应"""
        if status_code >= 400:
            raise self.error_handler.handle_http_error(status_code, response_text)
        
        try:
            return json.loads(response_text)
        except json.JSONDecodeError:
            return response_text
    
    @with_retry()
    def get(self, endpoint: str, params: Dict = None, **kwargs) -> Any:
        """发送GET请求"""
        url = f"{self.base_url}/{endpoint.lstrip('/')}"
        headers = self._prepare_headers(kwargs.get('headers'))
        
        # 模拟请求
        print(f"   发送GET请求: {url}")
        print(f"   请求头: {headers}")
        if params:
            print(f"   请求参数: {params}")
        
        # 模拟响应
        if "protected" in endpoint and not self.auth:
            return self._handle_response(401, '{"error": "需要认证"}')
        elif "rate-limit" in endpoint:
            return self._handle_response(429, '{"error": "请求过于频繁", "retry_after": 60}')
        elif "server-error" in endpoint:
            return self._handle_response(500, '{"error": "服务器内部错误"}')
        else:
            mock_data = {"message": "请求成功", "timestamp": datetime.now().isoformat()}
            return self._handle_response(200, json.dumps(mock_data, ensure_ascii=False))
    
    @with_retry()
    def post(self, endpoint: str, data: Dict = None, **kwargs) -> Any:
        """发送POST请求"""
        url = f"{self.base_url}/{endpoint.lstrip('/')}"
        headers = self._prepare_headers(kwargs.get('headers'))
        
        print(f"   发送POST请求: {url}")
        print(f"   请求数据: {data}")
        
        # 模拟创建资源
        if data:
            created_data = {"id": random.randint(100, 999), **data}
            return self._handle_response(201, json.dumps(created_data, ensure_ascii=False))
        else:
            return self._handle_response(400, '{"error": "请求数据为空"}')
    
    def set_rate_limit(self, requests_per_second: int):
        """设置速率限制"""
        self.rate_limit = requests_per_second
        self.last_request_time = 0
    
    def _enforce_rate_limit(self):
        """执行速率限制"""
        if hasattr(self, 'rate_limit'):
            current_time = time.time()
            time_since_last = current_time - self.last_request_time
            min_interval = 1.0 / self.rate_limit
            
            if time_since_last < min_interval:
                sleep_time = min_interval - time_since_last
                print(f"   速率限制: 等待 {sleep_time:.2f} 秒")
                time.sleep(sleep_time)
            
            self.last_request_time = time.time()

# API客户端使用示例
print(f"\n   API客户端使用示例:")

# 创建客户端
client = APIClient("https://api.example.com/v1")

# 测试无认证请求
try:
    response = client.get("public/data")
    print(f"   公共API响应: {response}")
except Exception as e:
    print(f"   请求失败: {e}")

# 测试需要认证的请求
try:
    response = client.get("protected/data")
except AuthenticationError as e:
    print(f"   认证错误: {e}")

# 使用认证的客户端
auth_client = APIClient(
    "https://api.example.com/v1",
    auth=BearerTokenAuth("your-bearer-token")
)

try:
    response = auth_client.get("protected/data")
    print(f"   认证API响应: {response}")
except Exception as e:
    print(f"   请求失败: {e}")

# 测试POST请求
try:
    new_data = {"name": "测试项目", "value": 123}
    response = auth_client.post("items", data=new_data)
    print(f"   创建结果: {response}")
except Exception as e:
    print(f"   创建失败: {e}")

# 测试限流处理
try:
    response = client.get("rate-limit/test")
except RateLimitError as e:
    print(f"   限流错误: {e}, {e.retry_after}秒后重试")

print(f"\n✅ API认证与错误处理完成")
print(f"🎯 学习目标达成:")
print(f"   ✓ 掌握常见的API认证方式")
print(f"   ✓ 理解网络异常的分类和处理")
print(f"   ✓ 能够实现重试机制和限流策略")
print(f"   ✓ 掌握构建健壮API客户端的技巧")

## 📋 学习总结

### ✅ 知识清单达成情况验证

**4.5 网络请求处理 [⭐⭐进阶]**
- ✅ 掌握HTTP协议的基本概念和方法
- ✅ 熟练使用requests库进行各种HTTP操作
- ✅ 理解API认证和会话管理机制
- ✅ 能够处理网络异常、重试和限流问题
- ✅ 掌握urllib和requests库的核心功能
- ✅ 理解RESTful API的交互模式
- ✅ 能实现完整的API客户端类
- ✅ 掌握认证、错误处理和重试机制
- ✅ 能独立构建稳定的网络应用

### 🎯 与LangChain学习的关联

**网络请求重要性**：
- 网络请求是LangChain API集成的基础
- HTTP客户端模式影响LangChain的LLM API调用
- 认证和错误处理对LangChain的稳定性至关重要
- 限流和重试机制在LangChain的API工具中发挥重要作用

**实际应用场景**：
- LangChain的LLM集成需要稳定的HTTP客户端
- API工具和函数调用依赖网络请求处理
- 外部数据源集成需要认证和错误处理
- 流式响应和异步请求在LangChain中广泛应用

### 📚 进阶学习建议

1. **练习建议**：
   - 多练习不同API的集成和认证方式
   - 深入学习异步HTTP请求处理
   - 掌握WebSocket和实时通信技术

2. **扩展学习**：
   - 学习HTTP/2和HTTP/3协议特性
   - 了解GraphQL和REST API的差异
   - 探索微服务架构中的服务通信

3. **实际应用**：
   - 构建API网关和代理服务
   - 开发分布式系统的通信层
   - 实现云服务的集成工具

### 🔧 常见错误与注意事项

1. **认证信息泄露**：
   ```python
   # 错误：硬编码认证信息
   api_key = "sk-1234567890abcdef"  # 不要这样做
   
   # 正确：使用环境变量或配置文件
   import os
   api_key = os.getenv('API_KEY')
   ```

2. **忽略HTTPS验证**：
   ```python
   # 错误：在生产环境忽略SSL验证
   response = requests.get(url, verify=False)
   
   # 正确：使用有效的证书或指定CA包
   response = requests.get(url, verify='/path/to/ca-bundle.crt')
   ```

3. **无限重试**：
   ```python
   # 错误：没有限制的重试
   while True:
       try:
           return make_request()
       except Exception:
           time.sleep(1)
   
   # 正确：使用指数退避和最大重试次数
   for attempt in range(max_retries):
       try:
           return make_request()
       except Exception as e:
           if attempt == max_retries - 1:
               raise e
           time.sleep(2 ** attempt)
   ```

4. **阻塞主线程**：
   ```python
   # 错误：在主线程中进行长时间网络请求
   def handle_request():
       response = requests.get(url, timeout=300)  # 阻塞5分钟
       return response
   
   # 正确：使用异步请求或设置合理的超时
   def handle_request():
       response = requests.get(url, timeout=30)
       return response
   # 或使用 aiohttp 进行异步请求
   ```

5. **内存泄漏**：
   ```python
   # 错误：没有正确关闭连接
   session = requests.Session()
   # 使用session但没有关闭
   
   # 正确：使用上下文管理器
   with requests.Session() as session:
       response = session.get(url)
   ```

6. **忽略响应大小**：
   ```python
   # 错误：下载大文件时没有流式处理
   response = requests.get('https://example.com/large-file.zip')
   data = response.content  # 可能占用大量内存
   
   # 正确：使用流式下载
   with requests.get('https://example.com/large-file.zip', stream=True) as response:
       with open('file.zip', 'wb') as f:
           for chunk in response.iter_content(chunk_size=8192):
               f.write(chunk)
   ```

---

**🎉 恭喜完成网络请求处理学习！**

你已经掌握了Python网络编程的核心技能，能够熟练使用requests库进行HTTP操作，掌握了API认证、错误处理、重试机制等高级技术，为后续学习LangChain的API集成和工具调用奠定了坚实基础。

## 🚀 下一步学习预告

**继续第四节：常用标准库**：
- 4.6 多线程与多进程 [⭐⭐进阶]
- 4.7 数据序列化与存储 [⭐⭐进阶]
- 4.8 系统信息获取 [⭐⭐进阶]
- 4.9 常用工具模块 [⭐⭐进阶]

继续加油，完成Python标准库的学习！